In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt  # plotting 
import matplotlib.font_manager as fm # to plot the font
from tqdm.auto import tqdm # see progress bar
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont # Darw picture from font
from skimage.transform import resize # Resizing of image
import cv2
from cv2 import resize as cv2_resize # resizng of image
from keras.preprocessing.image import ImageDataGenerator  # image augmentation on training images ONLY
from sklearn.model_selection import train_test_split  # splitting the data
import keras.backend as K # for custom metrices implementations and other processes that we define
from keras.layers import Dense,BatchNormalization,Input,Dropout,Conv2D,Flatten,MaxPool2D,LeakyReLU # keras layers
from keras.models import Model #Model class
from keras.optimizers import Adam #optimizer
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping 
# Call backs acts like milestones and if/else while model is being trained
import gc # garbage collector


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Someone in the Kaggle competition changed the parquet file to the feather file, which is more efficient.

Instructions to load the data:
* Click "+ Add Data" botton on right top.
* You can see "Search Datasets" text box on right top.
* Type "bengaliai-cv19-feather" to find the dataset.
* Press "Add" botton to add the data.

The credit is for [Kaggle public notebook: Bangali.AI super fast data loading with feather](http://www.kaggle.com/corochann/bangali-ai-super-fast-data-loading-with-feather?fbclid=IwAR2IqSN-JimwiSG5vfHxN5HTfwyYiywGu0kMKeqAc6KzxXrIbTPZrmGL1PE).

In [ ]:
path = "../input/bengaliaicv19feather"
train_path="/train_image_data_"
file_name=".feather"

In [ ]:
imagedata=pd.DataFrame()
for i in np.arange(4):
    imagedata=imagedata.append(pd.read_feather(path+train_path+str(i)+file_name).iloc[:,1:])

In [ ]:
imagedata.shape #200840 cases of training data and each image is 137*326=32332 pixels.

**DATA CLEANING**

There are 200840 training cases and not all of the cases are valid. Some of the images have only partial digit and some have interference term, some random dots.
We also need to resize the image and scale it to better fit our model.

1. Cleanup the images with cutoff.

2. Cleanup the images with random dots on the right hand side.

3. Resize the images.

4. Scale the images.



Here is some example about the cutoff images.

In [ ]:
plt.subplot(1,2,1)
plt.imshow(imagedata.iloc[16198,:].values.reshape(137,236))
plt.subplot(1,2,2)
plt.imshow(imagedata.iloc[196973,:].values.reshape(137,236))

There are some examples where there's random dots on the right hand side!  We should crop these out.

Use 1399 for thresholding later

In [ ]:
plt.imshow(imagedata.iloc[1399,:].values.reshape(137,236))

In [ ]:
#These will keep a list of inidices of which columns in our dataframe reprsent the leftmost and rightmost columns of our image.
left_list = []
left_5ov = []
right_list = []
right_5ov = []
indicies_to_remove = []

for i in range(0,(236*137)):
    if(i % 236 == 0):
        left_list.insert(-1,i)
    if(i % 236 == 235):
        right_list.insert(-1,i)
    if(i % 236 == 5):
        left_5ov.insert(-1,i)
    if(i % 236 == 230):
        right_5ov.insert(-1,i)
        

#We're checking to see which indicies have images that go off the screen so that we can remove them.
top_row = imagedata.iloc[:,0:236]
bot_row = imagedata.iloc[:,((236*137)-236):((236*137))]
left_col = imagedata.iloc[:,left_list]
left_5thcol = imagedata.iloc[:,left_5ov]
right_col = imagedata.iloc[:,right_list]
right_5thcol = imagedata.iloc[:,right_5ov]

#We stack the data so that we can quickly check if any values are 0.
top_row = top_row.stack().to_numpy()
bot_row = bot_row.stack().to_numpy()
left_col = left_col.stack().to_numpy()
left_5thcol = left_5thcol.stack().to_numpy()
right_col = right_col.stack().to_numpy()
right_5thcol = right_5thcol.stack().to_numpy()

#These are the rows in our dataset that we wish to remove.
indicies_to_remove.extend(np.floor(np.where(top_row < 100)[0]/236))
indicies_to_remove.extend(np.floor(np.where(bot_row < 100)[0]/236))
indicies_to_remove.extend(np.floor(np.where(left_col < 100)[0]/137))
indicies_to_remove.extend(np.floor(np.where(left_5thcol < 100)[0]/137))
indicies_to_remove.extend(np.floor(np.where(right_col < 100)[0]/137))
indicies_to_remove.extend(np.floor(np.where(right_5thcol < 100)[0]/137))

#Takes unique values
indicies_to_remove = np.unique(np.array(indicies_to_remove,dtype="int"))

In [ ]:
#How many bad examples are there?
len(indicies_to_remove)

In [ ]:
imagedata.index = [x for x in range(0, imagedata.shape[0])]

In [ ]:
path_1="../input/bengaliai-cv19/"
trainlabel=pd.read_csv(path_1+'train.csv')

In [ ]:
trainlabel=trainlabel.iloc[:,1:4]

In [ ]:
#Removes the bad images
imagedata = imagedata.iloc[~imagedata.index.isin(indicies_to_remove),:]

In [ ]:
trainlabel = trainlabel.iloc[~trainlabel.index.isin(indicies_to_remove),:]

In [ ]:
imagedata.shape

In [ ]:
trainlabel.shape

In [ ]:
grapheme = pd.get_dummies(trainlabel['grapheme_root']).values 
vowel = pd.get_dummies(trainlabel['vowel_diacritic']).values   
consonant = pd.get_dummies(trainlabel['consonant_diacritic']).values

In [ ]:
del indicies_to_remove
del path_1
del path
del train_path
del file_name
gc.collect()

In [ ]:
print(f'Number of unique grapheme roots: {trainlabel["grapheme_root"].nunique()}')
print(f'Number of unique vowel diacritic: {trainlabel["vowel_diacritic"].nunique()}')
print(f'Number of unique consonant diacritic: {trainlabel["consonant_diacritic"].nunique()}')

Now, we need to resize the data and scale it.

In [ ]:
def resize_dataset(imagedata,height=137,width=236,new_h=100,new_w=100):
    
    #imput: 
        #imagedata: the dataset of flatten images in each row.
        #height,width: the original image height and width.
        #new_h,new_w: the height and width we want.
    
    new_row = new_h*new_w
    image_list = [cv2.resize(imagedata.iloc[i,:].to_numpy().reshape(height,width),dsize=(new_h,new_w),interpolation=cv2.INTER_CUBIC).reshape(1,new_row) for i in tqdm(range(imagedata.shape[0]))]
    #resize the images to the size we want.
    rescaled_data = pd.DataFrame(tqdm([e for sl in image_list for e in sl]))
    rescaled_data = rescaled_data/255
    #scaled the data to change 0-255 value into 0-1.
    
    del image_list
    gc.collect()
    
    return rescaled_data

In [ ]:
image_list = resize_dataset(imagedata)

In [ ]:
del imagedata
gc.collect()    

In [ ]:
image_list.shape

**Build The Model**

* Add layers
* Define callbacks.
* Split the dataset.
* Fit the model

In [ ]:
# Initualize variables.
BATCH_SIZE = 150
final_w,final_h=100,100

Add layers

In [ ]:
inputs = Input(shape = (final_w, final_h, 1)) 
# Input layer takes the size of the input into consideration.

model = Conv2D(filters=64, kernel_size=(3,3), padding='SAME', activation='relu')(inputs)  
# layer using 64 as filter and a 3*3 matrix as kernal, 
# padding using 'SAME' means same padding in each side of the image,
# using relu as the activation function.
model = Conv2D(filters=64, kernel_size=(3,3), padding='SAME')(model)
model = LeakyReLU(alpha=0.17)(model) 
# using LeakyReLU as one independent layer, it is the same as using activation.
model = Conv2D(filters=64, kernel_size=(3,3), padding='SAME')(model)
model = LeakyReLU(alpha=0.13)(model)
model = BatchNormalization(momentum=0.15)(model)
# choose 0.15 as momentum to do normalization.
model = MaxPool2D(pool_size=(2, 2))(model) 
# find the max number in a 2*2 kernal.
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model) 
# drop out 30% weights to prevent overfitting.
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME')(model)
model = LeakyReLU()(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(3, 3))(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Dropout(rate=0.4)(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Flatten()(model) 
model = Dense(512)(model)

model = LeakyReLU()(model)
model = Dropout(rate=0.35)(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)
dense = Dense(186, activation = "relu")(model) 

out1 = Dense(168, activation = 'softmax',name='out_1')(dense) 
out2 = Dense(11, activation = 'softmax',name='out_2')(dense)  
out3 = Dense(7, activation = 'softmax',name='out_3')(dense)   

# 168,11,7 is the number of uniqueness of the three components.

model = Model(inputs=inputs, outputs=[out1,out2,out3]) 

#Finished by defining the Model's inputs and outputs.

Define the loss, optimizer, evaluation.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Define the callback.

In [ ]:
R_LR_P = ReduceLROnPlateau(monitor='val_out_1_loss',patience=3,verbose=1,factor=0.5,min_lr=0.00001)

ES = EarlyStopping(monitor='val_loss',patience=4, min_delta=0.0025,restore_best_weights=True)

MCP = ModelCheckpoint('best_model_weight.h5', monitor ='val_loss', verbose =1, 
                      save_best_only = True, save_weights_only=True)

callbacks = [R_LR_P,ES,MCP]

Define the Multi-classifier.

In [ ]:
class CustomDataGenerator(ImageDataGenerator):
    
    # extend the class ImageDataGenerator to CustomDataGenerator to change a single classifier to a multi classifier.
    
    def flow(self,x,y=None,batch_size=BATCH_SIZE,shuffle=True,sample_weight=None,seed=None,save_to_dir=None,
             save_prefix='',save_format='png',subset=None): 
        
        # Initialize
        labels_array = None 
        key_lengths = {} 
        ordered_labels = [] 
        
        for key, label_value in y.items():
            if labels_array is None:
                labels_array = label_value 
            else:
                labels_array = np.concatenate((labels_array, label_value), axis=1) 
                
            key_lengths[key] = label_value.shape[1] 
            ordered_labels.append(key)

        for x_out, y_out in super().flow(x, labels_array, batch_size=batch_size):
            label_dict = {} 
            i = 0 
            for label in ordered_labels:
                target_length = key_lengths[label]
                label_dict[label] = y_out[:, i: i + target_length] 
                i += target_length

            yield x_out, label_dict

Split the data

In [ ]:
image_list, x_test, y_train_graph, y_test_graph, y_train_vowel, y_test_vowel, y_train_conso, y_test_conso =\
    train_test_split(image_list, grapheme,vowel,consonant, test_size=0.15, random_state=13)
        # split the data in training and test to cross-validate our model's performance. 

In [ ]:
image_list=image_list.values.reshape(-1, final_w,final_h,1) # reshape into rank-4 matrix 
x_test=x_test.values.reshape(-1, final_w,final_h,1) 

In [ ]:
del grapheme
del vowel
del consonant
gc.collect()

Fit the model

In [ ]:
aug_generator = CustomDataGenerator(featurewise_center=False,samplewise_center=False,
                                    featurewise_std_normalization=False,samplewise_std_normalization=False,
                                    zca_whitening=False,rotation_range=13, zoom_range = 0.17,
                                    width_shift_range=0.18,height_shift_range=0.16,horizontal_flip=False,
                                    vertical_flip=False) 

In [ ]:
aug_generator.fit(image_list)

In [ ]:
history = model.fit_generator(aug_generator.flow(image_list, {'out_1': y_train_graph, 'out_2': y_train_vowel, 
                                                     'out_3': y_train_conso}, batch_size=BATCH_SIZE),
                              epochs = 27, validation_data = (x_test, [y_test_graph, y_test_vowel, y_test_conso]),
                              steps_per_epoch=image_list.shape[0] //BATCH_SIZE ,callbacks=callbacks)

# fit the model and train the model.

In [ ]:
stopped_at = ES.stopped_epoch # shows at which epoch model stopped due to early stopping

In [ ]:
del image_list
del x_test
del y_train_graph
del y_test_graph
del y_train_vowel
del y_test_vowel
del y_train_conso
del y_test_conso
gc.collect()

Apply to predict

In [ ]:
model.load_weights('best_model_weight.h5') 

# load the best weights 

In [ ]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}
# initialize

In [ ]:
path = "../input/bengaliaicv19feather"
test_path="/test_image_data_"
file_name=".feather"
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for j in range(4):
    print('Processing file %s'%j)
    df = pd.read_feather(path+test_path+str(j)+file_name)
    df.set_index('image_id', inplace=True)
    index_values = df.index.values

    df = resize_dataset(df,curr_w=137,curr_h=236,res_w=final_w,res_h=final_h).astype('float32')
    df = df.values.reshape(-1, final_w, final_h, 1)
    
    preds = model.predict(df)

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(index_values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    
    del df
    gc.collect()
    
# using the model to predict the test data.

In [ ]:
submission = pd.DataFrame({'row_id': row_id,'target':target},columns = ['row_id','target'])
submission.to_csv('submission.csv',index=False)
submission.head()
# define how to submit prediction.